In [1]:
pip install ucimlrepo


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
statlog_heart = fetch_ucirepo(id=145) 
  
# data (as pandas dataframes) 
X = statlog_heart.data.features 
y = statlog_heart.data.targets 
  
# metadata 
print(statlog_heart.metadata) 
  
# variable information 
print(statlog_heart.variables) 


{'uci_id': 145, 'name': 'Statlog (Heart)', 'repository_url': 'https://archive.ics.uci.edu/dataset/145/statlog+heart', 'data_url': 'https://archive.ics.uci.edu/static/public/145/data.csv', 'abstract': 'This dataset is a heart disease database similar to a database already present in the repository (Heart Disease databases) but in a slightly different form', 'area': 'Health and Medicine', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 270, 'num_features': 13, 'feature_types': ['Categorical', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['heart-disease'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': None, 'last_updated': 'Fri Feb 09 2024', 'dataset_doi': '10.24432/C57303', 'creators': [], 'intro_paper': None, 'additional_info': {'summary': 'Cost Matrix\r\n\r\n_______\t abse  pres\r\nabsence\t 0\t1\r\npresence  5\t0\r\n\r\nwhere the rows represent the true values and the columns the p

Feature Scaling and the preprocessing

In [3]:
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Fetch dataset
statlog_heart = fetch_ucirepo(id=145)
X = statlog_heart.data.features
y = statlog_heart.data.targets['heart-disease'].replace({1: 0, 2: 1})  # 0 = absence, 1 = presence

# Identify columns
numeric_features = ['age', 'rest-bp', 'serum-chol', 'max-heart-rate', 'oldpeak', 'major-vessels']
categorical_features = ['sex', 'fasting-blood-sugar', 'angina', 'chest-pain', 'electrocardiographic', 'slope', 'thal']

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(drop='first'), categorical_features)
    ])


Logistic Regression

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

# Full pipeline with logistic regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000, class_weight={0: 1, 1: 5}))
])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model
pipeline.fit(X_train, y_train)

# Predict
y_pred = pipeline.predict(X_test)

# Evaluate
print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.70      0.81        33
           1       0.67      0.95      0.78        21

    accuracy                           0.80        54
   macro avg       0.81      0.82      0.80        54
weighted avg       0.84      0.80      0.80        54

Confusion Matrix:
[[23 10]
 [ 1 20]]


Confusion matrix elaboration: 
True negatives (TN): 23 people without heart disease correctly predicted.

False positives (FP): 10 people without heart disease incorrectly predicted as having it.

False negatives (FN): 1 person with heart disease was missed — this is very important due to the cost.

True positives (TP): 20 people with heart disease correctly predicted.

There was only 1 false negative which is a very desirable outcome. 

The model correctly found 95% of the people who actually had heart disease.

However, about 1 out of every 3 people the model said had heart disease actually didn’t (false alarms).

Overall, the model was right 80% of the time, which is pretty good for a health-related prediction.

Feature importance

In [5]:

feature_names = (
    numeric_features +
    list(pipeline.named_steps['preprocessor'].transformers_[1][1].get_feature_names_out(categorical_features))
)

# Match features to coefficients
coefs = pipeline.named_steps['classifier'].coef_[0]
feature_importance = pd.Series(coefs, index=feature_names).sort_values(key=abs, ascending=False)

print("Top contributing features:")
print(feature_importance.head(10))


Top contributing features:
sex_1.0                    1.758673
chest-pain_4.0             1.686979
thal_7.0                   1.170336
fasting-blood-sugar_1.0   -0.811079
slope_2.0                  0.722755
major-vessels              0.625400
oldpeak                    0.604831
serum-chol                 0.421465
slope_3.0                  0.391631
rest-bp                    0.347952
dtype: float64
